In [18]:
import pandas as pd
import json
from tqdm import tqdm
import os
from datetime import datetime
from peewee import chunked, IntegrityError, MySQLDatabase, Model, CharField, DateTimeField, FloatField, IntegerField, BooleanField, TextField

In [19]:
class GameData(Model):
    AppID = IntegerField(primary_key=True)
    Name = CharField()
    ReleaseDate = DateTimeField(null=True)
    RequiredAge = IntegerField()
    Price = FloatField()
    DetailedDescription = TextField(null=True)
    AboutGame = TextField(null=True)
    ShortDescription = TextField(null=True)
    Reviews = TextField(null=True)
    Website = CharField(null=True)
    SupportURL = CharField(null=True)
    SupportEmail = CharField(null=True)
    HeaderImage = CharField(null=True)
    Windows = BooleanField()
    Mac = BooleanField()
    Linux = BooleanField()
    MetacriticScore = IntegerField()
    MetacriticURL = CharField(null=True)
    Achievements = IntegerField()
    Recommendations = IntegerField()
    Notes = TextField(null=True)
    SupportedLanguages = TextField(null=True)  # Stored as comma-separated
    FullAudioLanguages = TextField(null=True)  # Stored as comma-separated
    Developers = TextField(null=True)  # Stored as comma-separated
    Publishers = TextField(null=True)  # Stored as comma-separated
    Categories = TextField(null=True)  # Stored as comma-separated
    Genres = TextField(null=True)  # Stored as comma-separated
    Screenshots = TextField(null=True)  # Stored as comma-separated URLs
    Movies = TextField(null=True)  # Stored as comma-separated URLs
    UserScore = FloatField()
    ScoreRank = CharField(null=True)
    PositiveReviews = IntegerField()
    NegativeReviews = IntegerField()
    EstimatedOwnersMin = IntegerField()
    EstimatedOwnersMax = IntegerField()
    AvgPlaytimeForever = IntegerField()
    AvgPlaytimeTwoWeeks = IntegerField()
    MedianPlaytimeForever = IntegerField()
    MedianPlaytimeTwoWeeks = IntegerField()
    PeakCCU = IntegerField()
    Tags = TextField(null=True)  # Stored as JSON string of tag:count pairs
    CreatedAt = DateTimeField(default=datetime.now)
    UpdatedAt = DateTimeField(default=datetime.now)

    class Meta:
        database = None
        table_name = 'games'  # This explicitly sets the table name to 'games'

def get_database_connection(host, port, user, password, database):
    return MySQLDatabase(
        database,
        host=host,
        port=port,
        user=user,
        password=password
    )

def process_owners_range(owners_str):
    if not owners_str:
        return 0, 0
    try:
        min_str, max_str = owners_str.replace(',', '').split(' - ')
        return int(min_str), int(max_str)
    except:
        return 0, 0

def process_and_insert_node(data_dict, db):
    GameData._meta.database = db
    
    if db.is_closed():
        db.connect()
    
    db.create_tables([GameData])

    games_to_insert = []
    for app_id, game in tqdm(data_dict.items(), desc="Processing games"):
        try:
            # Process owners range
            owners_min, owners_max = process_owners_range(game.get('estimated_owners', '0 - 0'))
            
            # Convert lists to comma-separated strings
            supported_languages = ','.join(game.get('supported_languages', []))
            full_audio_languages = ','.join(game.get('full_audio_languages', []))
            developers = ','.join(game.get('developers', []))
            publishers = ','.join(game.get('publishers', []))
            categories = ','.join(game.get('categories', []))
            genres = ','.join(game.get('genres', []))
            screenshots = ','.join(game.get('screenshots', []))
            movies = ','.join(game.get('movies', []))
            
            # Convert tags dictionary to JSON string
            tags_json = json.dumps(game.get('tags', {}))

            games_to_insert.append({
                'AppID': int(app_id),
                'Name': game.get('name', ''),
                'ReleaseDate': pd.to_datetime(game.get('release_date')),
                'RequiredAge': game.get('required_age', 0),
                'Price': game.get('price', 0.0),
                'DetailedDescription': game.get('detailed_description', ''),
                'AboutGame': game.get('about_the_game', ''),
                'ShortDescription': game.get('short_description', ''),
                'Reviews': game.get('reviews', ''),
                'Website': game.get('website', ''),
                'SupportURL': game.get('support_url', ''),
                'SupportEmail': game.get('support_email', ''),
                'HeaderImage': game.get('header_image', ''),
                'Windows': game.get('windows', False),
                'Mac': game.get('mac', False),
                'Linux': game.get('linux', False),
                'MetacriticScore': game.get('metacritic_score', 0),
                'MetacriticURL': game.get('metacritic_url', ''),
                'Achievements': game.get('achievements', 0),
                'Recommendations': game.get('recommendations', 0),
                'Notes': game.get('notes', ''),
                'SupportedLanguages': supported_languages,
                'FullAudioLanguages': full_audio_languages,
                'Developers': developers,
                'Publishers': publishers,
                'Categories': categories,
                'Genres': genres,
                'Screenshots': screenshots,
                'Movies': movies,
                'UserScore': game.get('user_score', 0.0),
                'ScoreRank': game.get('score_rank', ''),
                'PositiveReviews': game.get('positive', 0),
                'NegativeReviews': game.get('negative', 0),
                'EstimatedOwnersMin': owners_min,
                'EstimatedOwnersMax': owners_max,
                'AvgPlaytimeForever': game.get('average_playtime_forever', 0),
                'AvgPlaytimeTwoWeeks': game.get('average_playtime_2weeks', 0),
                'MedianPlaytimeForever': game.get('median_playtime_forever', 0),
                'MedianPlaytimeTwoWeeks': game.get('median_playtime_2weeks', 0),
                'PeakCCU': game.get('peak_ccu', 0),
                'Tags': tags_json
            })
        except Exception as e:
            print(f"Error processing AppID {app_id}: {e}")

    if games_to_insert:
        with db.atomic():
            for batch in chunked(games_to_insert, 100):
                try:
                    GameData.insert_many(batch).execute()
                except IntegrityError as e:
                    print(f"Duplicate entry found, skipping batch: {e}")

    db.close()

In [22]:
# Load and process portion of data
with open('data/games.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Get the first quarter of the data
quarter_size = len(data) // 16
keys = list(data.keys())[:quarter_size]
data_subset = {k: data[k] for k in keys}

# Node distribution:
# Node 1: All data
# Node 2: Windows only
# Node 3: Windows + (Mac or Linux)
data_node1 = data_subset  # All data
data_node2 = {k: v for k, v in data_subset.items() 
              if v.get('windows') is True 
              and v.get('mac') is False 
              and v.get('linux') is False}  # Windows-only games
data_node3 = {k: v for k, v in data_subset.items() 
              if v.get('windows') is True 
              and (v.get('mac') is True or v.get('linux') is True)}  # Multi-platform games with Windows

# Print distribution stats
print(f"Total records in dataset: {len(data_subset)}")
print(f"Records in Node 1 (all): {len(data_node1)}")
print(f"Records in Node 2 (windows-only): {len(data_node2)}")
print(f"Records in Node 3 (windows+mac/linux): {len(data_node3)}")

# Process Node 1 - All games
db_node1 = get_database_connection(
    host='localhost',
    port=3306,
    user='admin',
    password='password',
    database='games'
)
process_and_insert_node(data_node1, db_node1)

# Process Node 2 - Windows-only games
db_node2 = get_database_connection(
    host='localhost',
    port=3307,
    user='admin',
    password='password',
    database='games'
)
process_and_insert_node(data_node2, db_node2)

db_node3 = get_database_connection(
    host='localhost',
    port=3308,
    user='admin',
    password='password',
    database='games'
)
process_and_insert_node(data_node3, db_node3)

# Close all connections
db_node1.close()
db_node2.close()
db_node3.close()

Total records in dataset: 6088
Records in Node 1 (all): 6088
Records in Node 2 (windows-only): 4543
Records in Node 3 (windows+mac/linux): 1538


Processing games: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1538/1538 [00:00<00:00, 2951.71it/s]


False